Resultados de la primera vuelta

In [1]:
import pandas as pd
import numpy as np
import pyreadstat as pyd

In [2]:
# datos de la primera vuelta electoral
primeravuelta, meta1 = pyd.read_sav('Data/Primera-Vuelta.sav', apply_value_formats=True)

In [3]:
primeravuelta.columns

Index(['DIGNIDAD_CODIGO', 'DIGNIDAD_NOMBRE', 'PROVINCIA_CODIGO',
       'PROVINCIA_NOMBRE', 'CIRCUNSCRIPCION_CODIGO', 'CIRCUNSCRIPCION_NOMBRE',
       'CANTON_CODIGO', 'CANTON_NOMBRE', 'PARROQUIA_CODIGO',
       'PARROQUIA_NOMBRE', 'JUNTA_SEXO', 'OP_CODIGO', 'CANDIDATO_NOMBRE',
       'SUFRAGANTES', 'BLANCOS', 'NULOS', 'VOTOS'],
      dtype='object')

In [4]:
# Ajustar tipo de datos - convertir float primero, luego int
primeravuelta = primeravuelta.astype({
    'DIGNIDAD_NOMBRE': str,
    'PROVINCIA_CODIGO': float,  # Primero a float
    'PROVINCIA_NOMBRE': str,
    'CIRCUNSCRIPCION_CODIGO': str,
    'CIRCUNSCRIPCION_NOMBRE': str,
    'CANTON_CODIGO': float,  # Primero a float
    'CANTON_NOMBRE': str, 
    'PARROQUIA_CODIGO': float,  # Primero a float
    'PARROQUIA_NOMBRE': str,
    'JUNTA_SEXO': str,
    'OP_CODIGO': float,  # Primero a float
    'SUFRAGANTES': float, 
    'BLANCOS': float,
    'NULOS': float, 
    'VOTOS': float,
    'CANDIDATO_NOMBRE': str
})

# Luego convertir a int (sin decimales)
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'OP_CODIGO']] = \
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'OP_CODIGO']].astype(int)

In [5]:
primeravuelta.head(5)

,DIGNIDAD_CODIGO,DIGNIDAD_NOMBRE,PROVINCIA_CODIGO,PROVINCIA_NOMBRE,CIRCUNSCRIPCION_CODIGO,CIRCUNSCRIPCION_NOMBRE,CANTON_CODIGO,CANTON_NOMBRE,PARROQUIA_CODIGO,PARROQUIA_NOMBRE,JUNTA_SEXO,OP_CODIGO,CANDIDATO_NOMBRE,SUFRAGANTES,BLANCOS,NULOS,VOTOS
0,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,7,LEONIDAS IZA,9755.0,188.0,804.0,574.0
1,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,18,HENRY CUCALON,9755.0,188.0,804.0,39.0
2,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,32,HENRY KRONFLE KOZHAYA,9755.0,188.0,804.0,34.0
3,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,34,ANDREA GONZALEZ,9755.0,188.0,804.0,433.0
4,11.0,PRESIDENTE Y VICEPRESIDENTE,1,AZUAY,0.0,nan,260,CUENCA,285,BAÑOS,FEMENINO,39,PEDRO GRANJA,9755.0,188.0,804.0,56.0


In [6]:
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'PARROQUIA_NOMBRE']].head(5)

,PROVINCIA_CODIGO,CANTON_CODIGO,PARROQUIA_CODIGO,PARROQUIA_NOMBRE
0,1,260,285,BAÑOS
1,1,260,285,BAÑOS
2,1,260,285,BAÑOS
3,1,260,285,BAÑOS
4,1,260,285,BAÑOS


In [7]:
dic = pd.read_excel('data/diccionario_parroquias.xlsx')

In [8]:
# Renombrar columnas para que coincidan con primeravuelta
dic = dic.rename(columns={'CODPRO': 'PROVINCIA_CODIGO',
                          'CODCAN': 'CANTON_CODIGO',
                          'CODPAR': 'PARROQUIA_CODIGO'}
                ).astype({'PROVINCIA_CODIGO': int, 
                          'CANTON_CODIGO': int,
                          'PARROQUIA_CODIGO': int
                          })

In [9]:
primeravuelta = primeravuelta.merge(
    dic,
    on = ['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO'], how='left'
    )

De momento, exploraremos solo los votos presidenciales

In [10]:
# Filtrar datos para PRESIDENTE Y VICEPRESIDENTE
presidente1 = primeravuelta[primeravuelta['DIGNIDAD_NOMBRE'] == 'PRESIDENTE Y VICEPRESIDENTE']

Los resultados de sufragantes, nulos y blancos solo pueden obtenerse a nivel de parroquia y junta de sexo. Además, hay que recordar que como el mapa del CNE es más viejo, hay que tratar la base para luego homologarla con el censo del 2022.

Observa el ejemplo siguiente.

In [11]:
# Así deben desagregarse los datos
presidente1[presidente1['ADM3_PCODE'] == 'EC010150']\
    .groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO'], as_index=False)\
    .agg(Sufragantes=('SUFRAGANTES', 'mean'),
         Votos_válidos=('VOTOS', 'sum'),
         Nulos=('NULOS', 'mean'),
         Blancos=('BLANCOS', 'mean')
        )

,ADM3_PCODE,PARROQUIA_NOMBRE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos
0,EC010150,MACHANGARA,FEMENINO,4831.0,4492.0,288.0,48.0
1,EC010150,MACHANGARA,MASCULINO,4184.0,3877.0,241.0,63.0


In [12]:
# exportar para verificar sumas:
# presidente1[presidente1['PARROQUIA_NOMBRE'] == 'CAÑARIBAMBA'].to_excel("data/comprobaciones/canaribamba.xlsx", index=False)

Aun así hay leves descuadres en las sumas de válidos, blancos y nulos. Pero eso ya es cuestión de la fuente. De momento, vamos a tratar los datos como sigue:

In [13]:
# Necesitamos un doble tratamiento, ya que ADM3_PCODE puede contener
# más de una parroquia lista en el CNE. (Esto puede comporbarse en la 
# unión espacial del cuaderno de trabajo homologar.ipynb).
# Así deben desagregarse los datos
resumen1 = presidente1\
                .groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO'], as_index=False)\
                .agg(Sufragantes=('SUFRAGANTES', 'mean'),
                    Votos_válidos=('VOTOS', 'sum'),
                    Nulos=('NULOS', 'mean'),
                    Blancos=('BLANCOS', 'mean')
                    )

resumen1 = resumen1.groupby(['ADM3_PCODE', 'JUNTA_SEXO'], as_index=False)\
    .agg(Sufragantes=('Sufragantes', 'sum'),
         Votos_válidos=('Votos_válidos', 'sum'),
         Nulos=('Nulos', 'mean'),
         Blancos=('Blancos', 'mean')
         )

resumen1.head(10)

,ADM3_PCODE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos
0,EC010150,FEMENINO,4831.0,4492.0,288.0,48.0
1,EC010150,MASCULINO,4184.0,3877.0,241.0,63.0
2,EC010151,FEMENINO,9755.0,8756.0,804.0,188.0
3,EC010151,MASCULINO,8294.0,7567.0,600.0,124.0
4,EC010152,FEMENINO,2349.0,2079.0,200.0,70.0
5,EC010152,MASCULINO,1724.0,1561.0,121.0,42.0
6,EC010153,FEMENINO,522.0,453.0,40.0,29.0
7,EC010153,MASCULINO,484.0,407.0,43.0,34.0
8,EC010154,FEMENINO,962.0,840.0,87.0,35.0
9,EC010154,MASCULINO,697.0,607.0,75.0,13.0


In [14]:
# Calcular porcentajes de nulos y blancos
resumen1['Nulos_pct'] = resumen1['Nulos'] / resumen1['Sufragantes']
resumen1['Blancos_pct'] = resumen1['Blancos'] / resumen1['Sufragantes']
resumen1.head(10)

,ADM3_PCODE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos,Nulos_pct,Blancos_pct
0,EC010150,FEMENINO,4831.0,4492.0,288.0,48.0,0.059615,0.009936
1,EC010150,MASCULINO,4184.0,3877.0,241.0,63.0,0.057600,0.015057
2,EC010151,FEMENINO,9755.0,8756.0,804.0,188.0,0.082419,0.019272
3,EC010151,MASCULINO,8294.0,7567.0,600.0,124.0,0.072341,0.014951
4,EC010152,FEMENINO,2349.0,2079.0,200.0,70.0,0.085143,0.029800
5,EC010152,MASCULINO,1724.0,1561.0,121.0,42.0,0.070186,0.024362
6,EC010153,FEMENINO,522.0,453.0,40.0,29.0,0.076628,0.055556
7,EC010153,MASCULINO,484.0,407.0,43.0,34.0,0.088843,0.070248
8,EC010154,FEMENINO,962.0,840.0,87.0,35.0,0.090437,0.036383
9,EC010154,MASCULINO,697.0,607.0,75.0,13.0,0.107604,0.018651


# Distribución de votos a candidatos por sufragante

In [15]:
# Agregamos clasificaciones ideológicas de candidatos
Political_Compass = pd.read_excel('data/PiliticalCompass.xlsx')
Political_Compass.head(3)

,Candidato,CANDIDATO_NOMBRE,Eje Económico (Izq–Der),Eje Social (Lib–Aut),Clasificación,Justificación breve,Separación
0,Luisa González (RC),LUISA GONZALEZ,–2.0,+1.5,Izquierda autoritaria,"Estado fuerte, redistribución, liderazgo centr...",LUISA GONZALEZ\n(Izquierda autoritaria)
1,Jorge Escala (Unidad Popular),JORGE ESCALA,–2.5,+2.0,Izquierda autoritaria,"Marxismo sindical, estatismo, orden desde movi...",Izquierda autoritaria
2,Pedro Granja (PSE),PEDRO GRANJA,–2.0,+1.0,Izquierda autoritaria,"Socialismo democrático radical, impuestos a gr...",Izquierda autoritaria


In [16]:
presidente1clas = presidente1.merge(Political_Compass[['CANDIDATO_NOMBRE', 'Separación']], on='CANDIDATO_NOMBRE', how='left')

In [17]:
# Hay que hacer un doble tratamiento igual que antes
distribucioncandidatos  = \
    presidente1clas.groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO', 'Separación'], 
                            as_index=False)\
                                .agg(Sufragantes=('SUFRAGANTES', 'mean'), 
                                     Votos_candidato=('VOTOS', 'sum'))

In [18]:
distribucioncandidatos[distribucioncandidatos['Separación'] == 'Centro']

,ADM3_PCODE,PARROQUIA_NOMBRE,JUNTA_SEXO,Separación,Sufragantes,Votos_candidato
0,EC010150,MACHANGARA,FEMENINO,Centro,4831.0,242.0
14,EC010150,MACHANGARA,MASCULINO,Centro,4184.0,194.0
28,EC010151,BAÑOS,FEMENINO,Centro,9755.0,433.0
42,EC010151,BAÑOS,MASCULINO,Centro,8294.0,372.0
56,EC010152,CUMBE,FEMENINO,Centro,2349.0,66.0
...,...,...,...,...,...,...
28854,EC240350,GRAL. A. E. GALLO,MASCULINO,Centro,938.0,18.0
28868,EC240351,ANCONCITO,FEMENINO,Centro,4647.0,85.0
28882,EC240351,ANCONCITO,MASCULINO,Centro,4426.0,55.0
28896,EC240352,JOSE LUIS TAMAYO / MUEY,FEMENINO,Centro,8505.0,253.0


Existen descuadres en los resultados. Habría que considerarlo.

In [19]:
distribucioncandidatos2  = \
    distribucioncandidatos.groupby(['ADM3_PCODE', 'Separación', 'JUNTA_SEXO'], 
                            as_index=False)\
                                .agg(Sufragantes=('Sufragantes', 'mean'), 
                                     Votos_candidato=('Votos_candidato', 'sum'))

In [20]:
distribucioncandidatos2.head()
# Comprobar con distribucioncandidatos2.to_excel("data/comprobaciones/distribucioncandidatos_ec010150_sexo.xlsx", index=False)
# Recuerda filtrar un ADM3_PCODE

,ADM3_PCODE,Separación,JUNTA_SEXO,Sufragantes,Votos_candidato
0,EC010150,Centro,FEMENINO,4831.0,242.0
1,EC010150,Centro,MASCULINO,4184.0,194.0
2,EC010150,Centro con tinte\ninstitucionalista,FEMENINO,4831.0,7.0
3,EC010150,Centro con tinte\ninstitucionalista,MASCULINO,4184.0,9.0
4,EC010150,Centro pragmático,FEMENINO,4831.0,16.0


In [21]:
distribucioncandidatos2['Votos_candidato_pct'] = distribucioncandidatos2['Votos_candidato'] / distribucioncandidatos2['Sufragantes']

In [22]:
distribucioncandidatos2.head(4)

,ADM3_PCODE,Separación,JUNTA_SEXO,Sufragantes,Votos_candidato,Votos_candidato_pct
0,EC010150,Centro,FEMENINO,4831.0,242.0,0.050093
1,EC010150,Centro,MASCULINO,4184.0,194.0,0.046367
2,EC010150,Centro con tinte\ninstitucionalista,FEMENINO,4831.0,7.0,0.001449
3,EC010150,Centro con tinte\ninstitucionalista,MASCULINO,4184.0,9.0,0.002151


In [23]:
reestructurado = distribucioncandidatos2.\
    pivot(index=['ADM3_PCODE', 'JUNTA_SEXO'], columns=['Separación'], 
          values='Votos_candidato_pct').fillna(0).reset_index()

In [24]:
# Unir tablas
table_resumen1 = resumen1.merge(reestructurado, on=['ADM3_PCODE', 'JUNTA_SEXO'], how='left').fillna(0)

In [25]:
# Separamos por sexo y exportamos (porque será una estructura más manejable en Tableau)
table_resumen1[table_resumen1['JUNTA_SEXO'] == 'MASCULINO'].to_csv('data/1raM.csv', index=False, decimal=',', sep=';')
table_resumen1[table_resumen1['JUNTA_SEXO'] == 'FEMENINO'].to_csv('data/1raF.csv', index=False, decimal=',', sep=';')
#table_resumen1.to_csv('data/ResultadosSegundaVuelta.csv', index=False, decimal=',', sep=';')